In [1]:
import os
import sys

MODULE_PATH = 'C:\Github\DL_Study\CNN'

sys.path.insert(0, MODULE_PATH)

In [2]:
# import
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy
import time

from ShuffleNet import *

# set random seed
torch.manual_seed(42)

In [3]:
def get_device():
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu')
    return device

def df_to_tensor(df):
    device = get_device()
    return torch.from_numpy(df.values).float().to(device)

def np_to_tensor(data):
    device = get_device()
    return torch.tensor(data).float().to(device)

# configuration setting
def model_config():
    # parameter for CNN Model
    epochs = [30]
    batch_size = [64]
    learning_rate = [0.01, 0.001]
    
    # create config data
    configs = []
    for i in epochs:
        for j in batch_size:
            for k in learning_rate:
                config = [i, j, k]
                configs.append(config)
    return configs

# fucntion for fit cnn model using configs
def model_fit(train_X, train_y, config, verbose=0):

    # unpack config
    n_epochs, n_batch, learning_rate = config
    # use ShuffleNet for CNN
    model = ShuffleNet(groups=3, in_channels=1)
    if torch.cuda.is_available():
        model.cuda()

    # define Loss and Optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    data_size = train_X.size(0)
    max_iters = data_size//n_batch

    for epoch in range(1, n_epochs+1):
        #shuffle data
        idx = numpy.random.permutation(numpy.arange(data_size))
        x_data = train_X[idx]
        y_data = train_y[idx]

        epoch_loss = 0
        start_time = time.time()
        for it in range(max_iters):
            batch_x = x_data[it*n_batch:(it+1)*n_batch]
            batch_y = y_data[it*n_batch:(it+1)*n_batch]

            optimizer.zero_grad()
            predict = model(batch_x)
            loss = criterion(predict, batch_y)
            loss.backward()
            optimizer.step()

            epoch_loss+= loss.item()
        avg_loss = epoch_loss/max_iters

        if verbose:
            duration = start_time-time.time()
            print(f'epoch:{epoch}/{epochs}, ì‹œê°„:{duration:.2f}[s], loss:{avg_loss:.5f}')


    return model

def MAE_metric(x, t):
    t = np.array(t)
    return np.mean(numpy.abs(x-t))

def MSE_metric(x, t):
    t = np.array(t)
    return np.mean((x-t)**2)

In [4]:
import zipfile, requests, io
import pandas as pd
import numpy as np
import numpy
import time
from datetime import datetime

numpy.random.seed(42)
np.random.seed(42)

data_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00374/'
data_name = 'energydata_complete.csv'
df = pd.read_csv(data_url+data_name)
df.drop(columns=df.columns[[0, 27, 28]], axis=1, inplace=True)
df.head()

,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,...,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint
0,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,...,18.2,48.900000,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3
1,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,...,18.2,48.863333,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2
2,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,...,18.2,48.730000,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1
3,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,...,18.1,48.590000,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0
4,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,...,18.1,48.590000,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9


In [5]:
# series data to img function
def series_to_img(dataset, time_step=1):
    num = dataset.shape[1]      # features num
    df = pd.DataFrame(dataset)
    cols, names = list(), list()
    # sequence t-n to t-1
    for i in range(time_step, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(num)]

    for i in range(0, 1):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(num)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(num)]

    agg = pd.concat(cols, axis=1)
    agg.columns = names
    agg.dropna(inplace=True)
    return agg

from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler

dataset = df.values
dataset = dataset.astype('float')

n_inputs = 24
n_features = 26
del_idx = n_inputs * n_features + 1
del_cols = [i for i in range(del_idx, del_idx+n_features-1)]
new_df = series_to_img(dataset, n_inputs)
new_df.drop(new_df.columns[del_cols], axis=1, inplace=True)

In [6]:
n_splits = 10
train_test_split = TimeSeriesSplit(n_splits=n_splits+1, gap=n_inputs).split(new_df)
next(train_test_split)

configs = model_config()
history = []

best_rmse, best_mse, best_mae = [], [], []
learning_time = []

i = 1

print('config : epochs, batch_size, learning_rate')

# nested cross validation for time series model
for train_cv_indices, test_cv_indices in train_test_split:
    print(f'fold : {i}/{n_splits}')
    i+=1

    # split x, y data
    train_cv_X, train_cv_y = new_df.iloc[train_cv_indices, :-1].values, new_df.iloc[train_cv_indices,-1].values
    test_cv_X, test_cv_y = new_df.iloc[test_cv_indices, :-1].values, new_df.iloc[test_cv_indices, -1].values

    # length for validation set
    test_length = int(len(train_cv_X)*0.2)

    # scaling data
    scaler_x = MinMaxScaler()
    train_cv_X = scaler_x.fit_transform(train_cv_X)
    test_cv_X = scaler_x.transform(test_cv_X)

    train_X, val_X = train_cv_X[:-test_length, :], train_cv_X[-test_length:, :]
    train_y, val_y = train_cv_y[:-test_length], train_cv_y[-test_length:]

    # reshape
    # inner loop
    train_X = train_X.reshape(-1, 1, n_inputs, n_features)
    val_X = val_X.reshape(-1, 1, n_inputs, n_features)
    train_y = train_y.reshape(-1, 1)
    val_y = val_y.reshape(-1, 1)

    # outer loop
    train_cv_X = train_cv_X.reshape(-1, 1, n_inputs, n_features)
    test_cv_X = test_cv_X.reshape(-1, 1, n_inputs, n_features)
    train_cv_y = train_cv_y.reshape(-1, 1)
    test_cv_y = test_cv_y.reshape(-1, 1)

    train_X = np_to_tensor(train_X)
    train_y = np_to_tensor(train_y)
    val_X = np_to_tensor(val_X)
    val_y = np_to_tensor(val_y)
    train_cv_X = np_to_tensor(train_cv_X)
    train_cv_y = np_to_tensor(train_cv_y)
    test_cv_X = np_to_tensor(test_cv_X)
    test_cv_y = np_to_tensor(test_cv_y)

    # model fit, inner
    errors = []
    for idx, cfg in enumerate(configs):
        print(f' == train {cfg} model == ', end=' ')
        model = model_fit(train_X, train_y, cfg)
        predicted = model(val_X)
        error = F.mse_loss(predicted, val_y)   # mse
        print(f'error(rmse):{np.sqrt(error.item()):.4f}')
        if errors:
            if error.item() < min(errors):
                param = idx
        else:
            param = idx
        errors.append(error.item())

    history.append(errors)

    # outer
    start_time = time.time()
    # model fitting
    selected_model = model_fit(train_cv_X,train_cv_y, configs[param])
    # check time
    duration = time.time() - start_time
    predicted = selected_model(test_cv_X)
    rmse = np.sqrt(F.mse_loss(predicted, test_cv_y).item())
    mse = F.mse_loss(predicted, test_cv_y)
    mae = F.l1_loss(predicted, test_cv_y)
    best_rmse.append(rmse)
    best_mse.append(mse.item())
    best_mae.append(mae.item())
    learning_time.append(duration)

    # model eval
    print(f'train-size:{train_X.size(0)}, val-size:{val_X.size(0)}, test-size:{test_cv_X.size(0)}')
    print(f'best_model => error(rmse) : {rmse:.2f}, param:{configs[param]}, times: {duration:.3f}')
    print()

config : epochs, batch_size, learning_rate
fold : 1/10
 == train [30, 64, 0.01] model ==  

C:\Users\lab\anaconda3\envs\pytorch\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


error(rmse):122.1371
 == train [30, 64, 0.001] model ==  error(rmse):137.4749
train-size:2614, val-size:653, test-size:1642
best_model => error(rmse) : 90.05, param:[30, 64, 0.01], times: 59.359

fold : 2/10
 == train [30, 64, 0.01] model ==  error(rmse):90.2360
 == train [30, 64, 0.001] model ==  error(rmse):112.4667
train-size:3928, val-size:981, test-size:1642
best_model => error(rmse) : 80.01, param:[30, 64, 0.01], times: 88.162

fold : 3/10
 == train [30, 64, 0.01] model ==  error(rmse):77.9547
 == train [30, 64, 0.001] model ==  error(rmse):98.7363
train-size:5241, val-size:1310, test-size:1642
best_model => error(rmse) : 77.12, param:[30, 64, 0.01], times: 118.493

fold : 4/10
 == train [30, 64, 0.01] model ==  error(rmse):74.6633
 == train [30, 64, 0.001] model ==  error(rmse):89.6073
train-size:6555, val-size:1638, test-size:1642
best_model => error(rmse) : 72.57, param:[30, 64, 0.01], times: 148.955

fold : 5/10
 == train [30, 64, 0.01] model ==  error(rmse):80.1655
 == train

In [8]:
def model_evaluation(mse, rmse, mae):
    mse = np.array(mse)
    rmse = np.array(rmse)
    mae = np.array(mae)
    print(f'MSE: mean={np.mean(mse)}, std={np.std(mse)}')
    print(f'RMSE: mean={np.mean(rmse)}, std={np.std(rmse)}')
    print(f'MAE: mean={np.mean(mae)}, std={np.std(mae)}')

model_evaluation(best_mse, best_rmse, best_mae)

# check time
print()
print('[training time]')
print(f'mean : {np.mean(np.array(learning_time))}, last:{learning_time[-1]}')

MSE: mean=5306.064672851562, std=1273.1959598459275
RMSE: mean=72.33815191284255, std=8.558998229119853
MAE: mean=34.82119941711426, std=6.2331326997678485

[training time]
mean : 193.28249373435975, last:328.08122730255127
